# Read in shapefile 

In [ ]:
import geopandas as gpd
import pandas as pd

# Read in the shapefile
gdf = gpd.read_file("C:\\PATH\\TO\\SHAPEFILE\\SHAPEFILE.shp")

# Convert the timestamp column to datetime type (change 't' to your own time column name)
gdf['t'] = pd.to_datetime(gdf['t'])

# select just 2019 data ## out if you want to download the entire dataset
gdf = gdf[gdf['t'].dt.year == 2019]

# Take only the first 10 rows (comment this out when you know the code is working)
gdf = gdf.head(10)

print("done")
print(len(gdf))

## Identify the Lat Long and time columns 

In [ ]:
gdf.columns

In [ ]:
# change 't' to time column in your dataset
timecolumn = 't'

# change geometry of polygon to the centroid 
# you can identify 
gdf['geometry'] = gdf['geometry'].centroid
gdf = gpd.GeoDataFrame(gdf, geometry='geometry')

gdf['Latitude'] = gdf['geometry'].y
gdf['Longitude'] = gdf['geometry'].x

## Downlaod data from http://thredds.northwestknowledge.net/
### see variables below

In [ ]:
import requests
import pandas as pd
import xarray as xr

url = ["http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_pr_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_rmax_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_rmin_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_sph_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_srad_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_th_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_tmmn_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_tmmx_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_vs_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_bi_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_fm100_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_fm1000_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_erc_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_pdsi_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_vpd_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_z_1979_CurrentYear_CONUS.nc"]


VariableList = [
    'precipitation_amount',
    'daily_maximum_relative_humidity',
    'daily_minimum_relative_humidity',
    'daily_mean_specific_humidity',
    'daily_mean_shortwave_radiation_at_surface',
    'daily_mean_wind_direction',
    'daily_minimum_temperature', 
    'daily_maximum_temperature', 
    'daily_mean_wind_speed',
    'daily_mean_burning_index_g',
    'dead_fuel_moisture_100hr',
    'dead_fuel_moisture_1000hr',
    'daily_mean_energy_release_component-g',
    'daily_mean_palmer_drought_severity_index', 
    'daily_mean_vapor_pressure_deficit',
    'daily_mean_palmer_z_index'
]


for variable, current_url in zip(VariableList, url):
    climate_values = []  # List to store extracted values for the current variable

    # Loop through the dataframe rows
    for idx, row in gdf.iterrows():
        # Extract the latitude, longitude, and date for the current row
        latitude = row["Latitude"]
        longitude = row["Longitude"]
        start_date = pd.to_datetime(row[timecolumn])

        # Set the request parameters
        params = {
            "var": variable,
            "latitude": latitude,
            "longitude": longitude,
            "time": start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
            "addLatLon": "true",
            "accept": "netcdf"
        }

        # Send the request
        response = requests.get(current_url, params=params, timeout=10)

        if response.status_code == 200:
            # If request is successful, save the data to a temporary netCDF file
            filename = f"temp_data_{variable}.nc"
            with open(filename, "wb") as f:
                f.write(response.content)

            # Load the netCDF data using xarray
            ds = xr.open_dataset(filename)

            # Extract the value and append to the list
            value_at_point = ds[variable].values[0]
            climate_values.append(value_at_point)
        else:
            print(f"Failed to retrieve data for index {idx} using variable {variable}. Status code:", response.status_code)
            climate_values.append(None)

        if (idx + 1) % 100 == 0:
            print(idx + 1)
            print(variable)

    # Append the climate_values list as a new column to gdf
    gdf[variable] = climate_values


In [ ]:
gdf